<h1>Jupyter ノートブックで BigQuery データを可視化する</h1>

BigQuery は、ペタバイト級のアナリティクス データ ウェアハウスであり、巨大容量のデータに対して SQL クエリをリアルタイムに近い速度で実行できます。

データ可視化ツールは、BigQuery データを把握して、インタラクティブに分析するのに役立ちます。可視化ツールを使って、傾向を特定して対応し、データを利用して予測を行うことができます。このチュートリアルでは、Jupyter ノートブックで BigQuery Python クライアント ライブラリと pandas を使用して、BigQuery の natality サンプル テーブルのデータを可視化します。

<h2>目標</h2>
このチュートリアルの内容は、次のとおりです。

- Jupyter ノートブックを実行する環境を設定する
- BigQuery Python クライアント ライブラリと pandas を使用して BigQuery データのクエリを実行し、可視化する

<h2>概要: Jupyter ノートブック</h2>

ノートブックにより、コードを作成および実行する環境が提供されます。ノートブックは、本質的にソース アーティファクトであり、`.ipynb` ファイルとして保存されます。このファイルには、説明テキスト コンテンツ、実行可能コードブロック、関連する結果（インタラクティブ HTML として表示される）を含めることができます。構造的に、ノートブックは一連のセルです。

セルは、結果を生成するために評価される入力テキストのブロックです。セルには次の 2 種類があります。

- コードセル - 評価するコードが含まれます。コードを実行して生成された出力や結果は、入力コードのすぐ後に表示されます。
- マークダウン セル - HTML に変換されるマークダウン テキストが含まれ、ヘッダー、リスト、書式付きテキストの生成に使用されます。

以下のスクリーンショットは、マークダウン セルとそれに続く Python コードセルを示しています。Python セルの出力はコードのすぐ下に表示されます。

In [ ]:
print('Hello World!')

開いた各ノートブックは、実行セッションに関連付けられます。IPython では、これはカーネルとも呼ばれます。このセッションは、ノートブック内に入力されたすべてのコードを実行し、状態（読み込まれる変数、その値、関数、クラス、既存の Python モジュール）を管理します。



<h2>BigQuery データのクエリと可視化</h2>

チュートリアルのこのセクションでは、BigQuery 内のデータのクエリと可視化に使用する Datalab ノートブックを作成します。[natality](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&hl=ja&_ga=2.238002882.336887729.1589073706-431044393.1585885350&_gac=1.196346334.1587955074.CjwKCAjw4pT1BRBUEiwAm5QuR4lYhZuwjdvkKXaQcmeIKwclqw8o3ZXPRBCThiC8R0r-EKgVvMDCJhoCbAwQAvD_BwE) サンプル テーブルのデータを使って、可視化を行います。このチュートリアルのクエリはすべて、[標準 SQL](https://cloud.google.com/bigquery/docs/reference/standard-sql/enabling-standard-sql?hl=ja) の構文を使用します。



BigQuery クライアント ライブラリにはセルマジックである %%bigquery が用意されています。<br>
このセルマジックは SQL クエリを実行して、結果を pandas DataFrame として返します。次のセルに以下のコードを入力すると、年別の出生数の合計が返されます。



In [ ]:
%%bigquery
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

次のセルブロックで以下のコマンドを入力して同じクエリを実行しますが、今回は新しい変数 total_births に結果を保存します（この変数は %%bigquery の引数として指定されています）。より詳細な分析と可視化のために、この結果を使用できます。

In [ ]:
%%bigquery total_births
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

これで、pandas DataFrame が変数 total_births に保存され、プロットに使用できます。クエリ結果のプロットを準備するには、次のセルに以下の組み込みマジック コマンドを貼り付けて matplotlib を有効にします。[matplotlib](http://matplotlib.org/) は、pandas で使用されるプロット用のライブラリです。

In [ ]:
%matplotlib inline

次のセルに以下のコードを入力し、pandas の DataFrame.plot() メソッドを使用してクエリ結果を棒グラフとして可視化します。<br>
pandas によるデータ可視化の詳細については、[pandas](https://pandas.pydata.org/pandas-docs/stable/visualization.html) のドキュメントをご覧ください。

In [ ]:
total_births.plot(kind='bar', x='year', y='birth_count');


続いて、次のセルに以下のクエリを貼り付けて、曜日別の出生数を取得します。

In [ ]:
%%bigquery births_by_weekday
SELECT
    wday,
    SUM(CASE WHEN is_male THEN 1 ELSE 0 END) AS male_births,
    SUM(CASE WHEN is_male THEN 0 ELSE 1 END) AS female_births
FROM `bigquery-public-data.samples.natality`
WHERE wday IS NOT NULL
GROUP BY wday
ORDER BY wday ASC

wday（weekday）フィールドでは null 値が許可されるため、このクエリでは wday が null であるレコードを除外しています。

次のセルに以下のコードを入力し、クエリ結果を折れ線グラフで表示します。



In [ ]:
births_by_weekday.plot(x='wday');

<h2>pandas DataFrame</h2>

マジック コマンドを使用すると、最小限の構文で BigQuery を操作できます。<br>
%%bigquery のバックグラウンドでは、BigQuery Python クライアント ライブラリを使用して指定のクエリを実行し、結果を pandas DataFrame に変換して、オプションで結果を変数に保存し、最後に結果を表示します。<br>
マジック コマンドの代わりに BigQuery Python クライアント ライブラリを直接使用すると、クエリをより詳細に制御でき、より複雑な構成が可能になります。<br>
ライブラリと pandas の統合により、宣言型の SQL の機能と命令コード（Python）を組み合わせて、データの分析、可視化、変換操作を行うことができます。



<h2>pandas DataFrame による BigQuery データのクエリと可視化</h2>

チュートリアルのこのセクションでは、pandas DataFrame を使用して BigQuery データのクエリ実行と可視化を行います。<br>
BigQuery Python クライアント ライブラリを使用して BigQuery データのクエリを実行します。pandas ライブラリの DataFrame を使用してデータを分析します。



In [ ]:
%%bigquery df

SELECT
    plurality,
    COUNT(1) AS count,
    year
FROM
    `bigquery-public-data.samples.natality`
WHERE
    NOT IS_NAN(plurality) AND plurality > 1
GROUP BY
    plurality, year
ORDER BY
    count DESC

DataFrame 内のクエリ結果をグラフ表示するには、次のセルに以下のコードを挿入して、データのピボットを行い、時系列の多生児数の積み上げ棒グラフを作成します。

In [ ]:
pivot_table = df.pivot(index='year', columns='plurality', values='count')
pivot_table.plot(kind='bar', stacked=True, figsize=(15, 7));

In [ ]:
%%bigquery df
SELECT
    gestation_weeks,
    COUNT(1) AS count
FROM
    `bigquery-public-data.samples.natality`
WHERE
    NOT IS_NAN(gestation_weeks) AND gestation_weeks <> 99
GROUP BY
    gestation_weeks
ORDER BY
    gestation_weeks

In [ ]:
ax = df.plot(kind='bar', x='gestation_weeks', y='count', figsize=(15,7))
ax.set_title('Count of Births by Gestation Weeks')
ax.set_xlabel('Gestation Weeks')
ax.set_ylabel('Count');

Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

source:[https://cloud.google.com/bigquery/docs/visualize-jupyter?hl=ja](https://cloud.google.com/bigquery/docs/visualize-jupyter?hl=ja)
